### ------------------------------------------------------------------------------------------------------------

In [ ]:
import os

if 'COLAB_GPU' in os.environ:
    !git clone https://github.com/impulsecorp/PickStocks.git
    !mv PickStocks/*.py .
    !mv PickStocks/data .
    !pip install -U -qq -r PickStocks/requirements.txt

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import system
from system import *
# small hack to prevent Colab error
try:
    from datablock import *
except:
    from datablock import *
seed

In [ ]:
%pylab inline

### Setup

In [ ]:
data_timeperiod = 'D'
data = get_data('SPY', period=data_timeperiod, nrows=None)
data = procdata_lite(data)

In [ ]:
# for inspectiion
print(data.shape)
data.head()

### ------------------------------------------------------------------------------------------------------------

### Parameters

In [ ]:
system.train_set_end = 0.5 # percentage point specifying the training set end point (1.0 means all data is training set)
system.val_set_end = 0.75    # percentage point specifying the validation set end point (1.0 means no test set)
system.balance_data = 1
system.scale_data = 1

system.multiclass = 0
system.multiclass_move_threshold = 1.0

combine_method = 'and' # whether to combine the strategies with 'and' or 'or'

individual_size = 3 # number of strategies to combine for each genome
eval_min_trades = 80 # minumum number of trades to be eligible for reproduction

In [ ]:
# EA Parameters
pop_size = 128
num_generations = 50
survival_rate = 0.2
crossover_prob = 0.5
mutation_prob = 0.5
mutation_prob_gene = 0.1
mutation_prob_gene_hard = 0.1

In [ ]:
def objective_1(a): # mean profit
    return float(np.mean(a['profit'].values))
def objective_2(a): # winner %
    return float(get_winner_pct(a))
def objective_3(a): # PF
    return float(get_profit_factor(a))
def objective_4(a): # num trades
    return float(len(a))

from empyrical import sortino_ratio, omega_ratio, sharpe_ratio, calmar_ratio, stability_of_timeseries
def obj_sortino(a):
    returns = a['profit'].values
    return float(sortino_ratio(returns))

# Combine one or more objectives here for the fitness function and set their relative weights
objectives = [

      (obj_sortino, 1.0),
    
    # (objective_1, 0.2),
    # (objective_2, 0.5),
    # (objective_3, 1.0),

              ]

### ------------------------------------------------------------------------------------------------------------

### Base classifier

In [ ]:
# Train LogisticRegression classifier on train data
clf, scaler = train_classifier(LogisticRegression, data)

In [ ]:
# Test on val data
equity, pf, base_trades = qbacktest(clf, scaler, data)

In [ ]:
base_trades.head()

### ------------------------------------------------------------------------------------------------------------

### Obtain all individual startegies (genes) to combine and their stats

In [ ]:
all_coords, feature_names, feat_bins, df = compute_feature_matrix(data, base_trades,
                                        min_pf=0.1,
                                        min_trades=10,
                                        max_trades=10000,
                                        topn=None)
df

### Evolutionary algorithm setup

In [ ]:
worst_possible_fitness = -999999.0

In [ ]:
def evaluate(genome):
    return fitness_function(get_genome_alltrades_binned(data, genome, base_trades, feature_names, feat_bins, combine_method=combine_method),
                            objectives = objectives,
                            eval_min_trades=eval_min_trades, worst_possible_fitness=worst_possible_fitness)

def eval_wrapper(ind_list): # wrapper for parallel processing
    return evaluate(ind_list)

In [ ]:
def create_individual():
    return rnd.sample(all_coords, individual_size)

def create_population(n):
    return [creator.Individual(create_individual()) for _ in range(n)]

def custom_mutation(individual):
    for i in range(len(individual)):
        if rnd.random() < mutation_prob_gene:
            if rnd.random() < mutation_prob_gene_hard:
                individual[i] = rnd.choice(all_coords)
            else:
                gene_row, gene_col = individual[i]
                if rnd.random() < 0.5:
                    gene_col = gene_col + 1
                else:
                    gene_col = gene_col - 1
                gene_col = np.clip(gene_col, 1, len(feat_bins[gene_row])-1)
                individual[i] = (gene_row, gene_col)

    return individual,

In [ ]:
creator.create("FitnessMax", base.Fitness, weights=list([x[1] for x in objectives]))
creator.create("Individual", list, fitness=creator.FitnessMax)
toolbox = base.Toolbox()
# Register individual and population creation functions
toolbox.register("individual", create_individual)
toolbox.register("population", create_population)
# Register genetic operators
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", custom_mutation)
toolbox.register("select", tools.selSPEA2)
# Register the fitness function
toolbox.register("evaluate", eval_wrapper)

### ------------------------------------------------------------------------------------------------------------

### Evolutionary algorithm

In [ ]:
best_genome = run_evolution(pop_size, toolbox, num_generations, survival_rate, crossover_prob, mutation_prob, objectives, worst_possible_fitness)

In [ ]:
# Description of the best strategy (feature name [ range where it can trade ])
for i in range(len(best_genome)):
    r,c = best_genome[i]
    print(feature_names[r], f'[{feat_bins[r][c-1]:.5f} .. {feat_bins[r][c]:.5f}]')

### ------------------------------------------------------------------------------------------------------------

### Test the best individual on val data

In [ ]:
alltrades = get_genome_alltrades_binned(data, best_genome, base_trades, feature_names, feat_bins, combine_method=combine_method)
plt.plot(alltrades['profit'].cumsum())
print(f'Profit factor: {get_profit_factor(alltrades):.5f}, Winners: {get_winner_pct(alltrades):.2f}%, Trades: {len(alltrades)}')

### Test the best individual on unseen data

In [ ]:
# Base test without the filter
*_, test_trades = qbacktest(clf, scaler, data, skip_val=1, skip_test=0, quiet=1)

In [ ]:
# Test with the filter
alltrades = get_genome_alltrades_binned(data, best_genome, test_trades, feature_names, feat_bins, combine_method=combine_method)
plt.plot(alltrades['profit'].cumsum())
print(f'Profit factor: {get_profit_factor(alltrades):.5f}, Winners: {get_winner_pct(alltrades):.2f}%, Trades: {len(alltrades)}')

In [ ]:
alltrades[0:20]

### ------------------------------------------------------------------------------------------------------------